In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm_notebook as tqdm

In [2]:
fpath = './Data/family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')

fpath = './submission_75955.csv'
submission = pd.read_csv(fpath, index_col='family_id')

In [3]:
N_DAYS = 100
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125


family_size = data.n_people.values
days_array = np.arange(N_DAYS, 0, -1)
choice_dict = data.loc[:, 'choice_0': 'choice_9'].T.to_dict()

choice_array_num = np.full((data.shape[0], N_DAYS + 1), -1)
for i, choice in enumerate(data.loc[:, 'choice_0': 'choice_9'].values):
    for d, day in enumerate(choice):
        choice_array_num[i, day] = d

best = submission['assigned_day'].values
        
penalties_array = np.array([
    [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n,
        200 + 18 * n,
        300 + 18 * n,
        300 + 36 * n,
        400 + 36 * n,
        500 + 36 * n + 199 * n,
        500 + 36 * n + 398 * n
    ]
    for n in range(family_size.max() + 1)
])

In [4]:
# Generating permutation using Heap Algorithm 
def heapPermutation(a, size, n): 
      
    # if size becomes 1 then prints the obtained 
    # permutation 
    if (size == 1): 
        yield a
  
    for i in range(size): 
        yield from heapPermutation(a,size-1,n); 
  
        # if size is odd, swap first and last 
        # element 
        # else If size is even, swap ith and last element 
        if size&1: 
            a[0], a[size-1] = a[size-1],a[0] 
        else: 
            a[i], a[size-1] = a[size-1],a[i] 

In [5]:
def checkIfDuplicates(listOfElems):
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True

In [6]:
def get_pref(x,p): 
    return int(x[1][:-1][x[1][:-1] == p].index.values[0].split('_')[1])


In [ ]:
NB_PERM = 3
fam_active_choice = np.empty(NB_PERM, dtype=int)
result = []


for x in tqdm(itertools.product(data.iterrows(), repeat=NB_PERM), total=5000**NB_PERM):
    if checkIfDuplicates([z[0] for z in x]):
        continue
    
    for n in range(NB_PERM):
        fam_active_choice[n] = best[x[n][0]]

    for p in heapPermutation(fam_active_choice.copy(), NB_PERM, NB_PERM):
        if  any([f == p for f, p in zip(fam_active_choice, p)]) or not all([pc in z[1][:-1].values for z,pc in zip(x,p)]) :
            continue 
        cost = 0
        for i in range(NB_PERM):
            fs = family_size[x[n][0]]
            cost += penalties_array[fs, get_pref(x[i], fam_active_choice[i])] - penalties_array[fs ,get_pref(x[i], p[i])]
        if cost > 0 :
            result.append({"cost":cost,"p": p , "fam_active_choice" : fam_active_choice, "fam_size" : fs})


In [ ]:
dt = pd.DataFrame(result)

print(dt["cost"].max())
val = 0 
for row in dt.iterrows():
    if row[1][0] > 0 :
       val += row[1][0]
print(val)
